# QuickMeet - Create List of Action Items from Meeting Transcriptions

```bash
Author: Han-Elliot Phan
Email: hanelliotphan@gmail.com

Last Update: February 11, 2025
```

## Brief Discussion
This project is to create list of action items based on the meeting transcriptions using Python and Llama-3 model. This notebook is used solely for research purpose. The source code of the project can be found [here](https://github.com/hanelliotphan/quick-meet).

## Architecture
This project uses the OpenAI's Whisper-1 model for audio-to-text transcription, and Meta's LLama-3 model for quantization for the process of generating list of action items.

For more information about the model, please read the following documentation
- [OpenAI's Whisper-1](https://platform.openai.com/docs/guides/speech-to-text)
- [Meta's Llama-3](https://huggingface.co/meta-llama/Llama-3.1-8B-Instruct)

## Dedication
I dedicate this hard-work commitment to myself, my mother, my best friend Ha-Phuong and those that have imprinted in my heart. I hope that I have made you all truly proud of me.

--------------------------------------------------------------------------------

In [1]:
!nvidia-smi

Wed Feb 12 01:29:16 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   72C    P8             11W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
!pip install -q -U requests torch torchvision bitsandbytes transformers accelerate sentencepiece datasets soundfile

In [3]:
# Import libraries

import os
import requests
import torch

from IPython.display import display, Markdown, update_display
from openai import OpenAI
from google.colab import drive, userdata
from huggingface_hub import login
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer, BitsAndBytesConfig

In [4]:
# Set up model constants

AUDIO_MODEL = "whisper-1"
LLAMA3 = "meta-llama/Meta-Llama-3.1-8B-Instruct"

In [5]:
# Fetch audio transcription
# Source: https://www.rmofspringfield.ca/p/meeting-audio-files

drive.mount("/content/drive")
meeting_audio_filepath = "/content/drive/MyDrive/ai-projects/quick-meet/meeting-audios/meeting-audio.mp4"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
# Sign in to HuggingFace

hf_token = userdata.get("HF_TOKEN")
login(token=hf_token)

In [7]:
# Sign in to OpenAI

openai_api_key = userdata.get("OPENAI_API_KEY")
openai = OpenAI(api_key=openai_api_key)

In [8]:
# Convert audio to text using OpenAI Whisper-1 model

meeting_audio = open(meeting_audio_filepath, "rb")
transcript = openai.audio.transcriptions.create(model=AUDIO_MODEL, file=meeting_audio, response_format="text")
print(transcript)

The planning meeting provides the opportunity for the command and general staff, as well as agency officials and other key personnel, to hear the complete plan for the next operational period as proposed by the Operations Section Chief. By the time the meeting occurs, the IMT should have vetted the information so there would be few, if any changes needed during the meeting, and no one is surprised by the proposed plan. The goal of the planning meeting is really to get the final buy-off from the incident management team members and some of the stakeholders, and that's really what it's all about, is getting that final support, that final buy-off, where people can walk away and say, this is implementable, it is supportable, and it's going to do the trick. I'd like to welcome everybody to the planning meeting for tonight's operational plan. If I could have everybody kind of check their cell phones, pagers, radios, and if you could kind of keep your side conversations to yourself, certainly

In [9]:
# Prompt to perform to-do generation

system_msg = "I am an assistant that takes in an audio transcript and then provides summary, key discussion points, takeaways and list of action items"
user_prompt = "Here is a meeting audio transcript. Please provide the location, date and summary of the meeting with attendees, and list of action items accoridingly: ", transcript

messages = [
    {"role": "system", "content": system_msg},
    {"role": "user", "content": user_prompt}
]

In [10]:
# Configure quantization

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [11]:
# Set up Llama-3 tokenizer

llama3_tokenizer = AutoTokenizer.from_pretrained(LLAMA3)
llama3_tokenizer.pad_token = llama3_tokenizer.eos_token
inputs = llama3_tokenizer.apply_chat_template(messages, return_tensors="pt").to("cuda")
streamer = TextStreamer(llama3_tokenizer)
model = AutoModelForCausalLM.from_pretrained(
    LLAMA3,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=quant_config
)
outputs = model.generate(inputs, streamer=streamer, max_new_tokens=2000)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

I am an assistant that takes in an audio transcript and then provides summary, key discussion points, takeaways and list of action items<|eot_id|><|start_header_id|>user<|end_header_id|>

('Here is a meeting audio transcript. Please provide the location, date and summary of the meeting with attendees, and list of action items accoridingly: ', "The planning meeting provides the opportunity for the command and general staff, as well as agency officials and other key personnel, to hear the complete plan for the next operational period as proposed by the Operations Section Chief. By the time the meeting occurs, the IMT should have vetted the information so there would be few, if any changes needed during the meeting, and no one is surprised by the proposed plan. The goal of the planning meeting is really to get the final buy-off from the incident management team membe

In [12]:
# Display Markdow version of the final script

final_script = llama3_tokenizer.decode(outputs[0], skip_special_tokens=True)
display(Markdown(final_script))

system

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

I am an assistant that takes in an audio transcript and then provides summary, key discussion points, takeaways and list of action itemsuser

('Here is a meeting audio transcript. Please provide the location, date and summary of the meeting with attendees, and list of action items accoridingly: ', "The planning meeting provides the opportunity for the command and general staff, as well as agency officials and other key personnel, to hear the complete plan for the next operational period as proposed by the Operations Section Chief. By the time the meeting occurs, the IMT should have vetted the information so there would be few, if any changes needed during the meeting, and no one is surprised by the proposed plan. The goal of the planning meeting is really to get the final buy-off from the incident management team members and some of the stakeholders, and that's really what it's all about, is getting that final support, that final buy-off, where people can walk away and say, this is implementable, it is supportable, and it's going to do the trick. I'd like to welcome everybody to the planning meeting for tonight's operational plan. If I could have everybody kind of check their cell phones, pagers, radios, and if you could kind of keep your side conversations to yourself, certainly appreciate it. With that, let's get going, and we'll start with an update on the current situation with our Operations Section Chief, Rich. Thank you, Joe. The incident we're working on started early this morning when a tractor-trailer on Highway 265 was making the transition onto Highway 71. In the process of coming off the transition road, it went off the road, overturned, and spilled its load of diazonon. The trailer was loaded with 15 300-gallon totes of diazonon, which spilled. We also have a spill of about 200 gallons of diesel fuel from the truck. The spill, the chemicals started down through this drainage here, went through a culvert underneath the freeway, and continued on. It currently has contaminated a couple of the water ponds along the drainage there, pond number two, and we're checking into contamination right now in pond number three. Driver had moderate injuries, was transported to one of the local hospitals. For scene control right now, we have six roadblocks set up on 265 and 71. Our biggest concern right now is that the chemicals will continue downstream here in this drainage and have an impact on two domestic water systems downstream. We have advised those water companies that the potential is there, and they're prepared to shut down those intakes. Thank you. Okay, we do have the incident objectives posted right here. They are the ones that were developed during the unified command meeting. Just confirming that they are still valid. Still valid. Yes. Yep, still valid. All right, sounds great. Let's get started about what's going to happen tonight. I've got the weather. Looks like this is going to be the calm before the storm essentially starting tonight. Coming from the southwest, we've got a fairly good storm coming in, so it'll be increasing clouds, possibility of one to two inches of rain, 68 degrees tonight. Winds are going to be out of the southwest, about five to 15. Looking towards the morning and throughout the day, eventually it'll be a high of 86. Winds again are going to be pretty much the same as far as the five to 15 from the southwest. So with that, to set up the operational plan, I'll have Rich come up and talk about what he's going to do tomorrow. Rich. Thanks again, Jeff. All right, we're going to continue with the organization as it's presently set up. We have two geographic divisions. We have Division A, Division B, and then we have three functional groups that we're going to operate through this operational period. In Division A, we're going to improve the dikes around the spill area. The Operations Section Chief will go over tactics, work assignments, and resources for each branch, division, and group on the ICS 215 wall chart. Any questions about the night operational plan? Thank you. All right. Thanks a lot, Rich. Next with a safety analysis, John, our safety officer. I've done an incident safety analysis based on the tactics as presented by the Operations Section Chief. I've discussed our mitigation measures with him, and we think we have a good opportunity to pursue this plan. Our primary concern is that we are dealing with a moderately toxic organophosphate pesticide, biazanone. It is also a combustible liquid, so we need to make sure that everybody's in proper PPE for the activities that are being accomplished. In the event that we do have fire in the product, we need to make sure we use as little water as possible in suppression because the runoff is a greater concern than is the smoke. We're also going to have an issue with fatigue and dehydration with our folks working in PPE, so we need to make sure that those folks are properly hydrated. I've ordered three additional safety officers, assistant safety officers, one for Division Alpha, one for the decon group, and one for the treatment group. Any questions on the safety plan? Thank you. All right. Thanks, John. Next, let's do some information sharing. Looks like logistics. Dale? Thanks, Jeff. I have a few things I need to cover. Food and drinks are on order along with tents for the Responder Rehab Area. We'll get that set up as soon as possible when it comes in. So far, the Emergency Operations Center has been able to fill orders when placed. All fresh resources will be here for the operations briefing, and we're setting up a briefing area north side of the Incident Command Post in the A Building Conference Room. Does anybody have any questions for logistics? Next, Jim. Liaison? I've attempted to contact every agency that's on the scene. If you're aware of any new agencies that are here, please let me know so I can find out if they're going to have a representative on site. Thank you. Thanks, Jim. Next, information. Carol? The local MEC has decided to set up a joint information center, and they're almost set up and ready to go. There's a lot of media interest, so please be patient. We will be presenting the first press release to the IC for approval, and I will have a message written up to put in the IAP also. Thank you. All right. Thanks, Carol. How about Bill? Finance? I don't have anything new, but we'll have a time unit leader and a cost unit leader here shortly so we can start tracking the expenditures. I do see that we do have an agency rep from the Department of Transportation, so Steve. Thank you. We do have a number of traffic control safety devices in route and in staging. When we're advised that you want to open the interstate, we'll be able to partially deploy those for opening the road. Excellent. Thank you. Okay. We've heard the plan and shared our information, so can we as a team support the plan looking at logistics? Support plan. Liaison? Information? Yes. Finance? Bill? Yes. Okay. And, of course, we need to have safety buyoff? Support the plan. Excellent. Okay. It looks like that's a wrap then. I also see that we do have our agency administrators here, and I'd certainly like to give them an opportunity to say something if they have. So let's start with Lucia. Thanks, Jeff. I just wanted to add that the County Emergency Operations Center director has briefed the State Department of Emergency Management, so they're aware of our situation. Thank you. I appreciate that. Joel? No comments. Okay. Thank you. Okay. Looking for concurrence from our Unified Command. Start with Ben. Yes. From the EPA's perspective, everything's looking pretty good. Our agency does have a representative en route to serve as a group supervisor, and they will be here in time for the ops briefing. Thanks, Ben. John? Yeah. I agree. I think we're right on track. As far as the state DEQ, very pleased with all the efforts for our actions so far. All right. Thank you very much. Okay. From the Jefferson County standpoint, I certainly concur with the plan. It appears to be a good, solid plan that everybody supports, so I want to thank everybody for pulling this thing together in a short period of time. We're going to be in pretty good shape for this upcoming operational period. The only other thing I would add is that I'll continue to be the Unified Incident Command spokesperson for the rest of this upcoming operational period, and then we'll hand that over to one of the other ICs tomorrow. That's all I've got. I'm going to go ahead and get ready for the briefing. All right. Thanks, Joe. Okay. I need the IAP components in to me by 1530 this afternoon to get ready for the 1800 operational briefing for tonight, and evidently, logistics is building that north of our command post, so you'll be reporting out there to do the operational briefing. We also have a 2000 planning meeting that will be back here to work on tomorrow's day operations. With that, please go ahead and turn your cell phones and pagers back on and get those IAP parts in to help me out. Thank you.\n")assistant

**Meeting Details:**

- **Location:** Incident Command Post, A Building Conference Room
- **Date:** Not specified in the transcript
- **Time:** Not specified in the transcript, but mentioned as "tonight" and "tomorrow"
- **Attendees:** 
  1. Jeff ( Incident Commander)
  2. Rich (Operations Section Chief)
  3. John (Safety Officer)
  4. Dale (Logistics)
  5. Jim (Liaison)
  6. Carol (Information)
  7. Bill (Finance)
  8. Steve (Department of Transportation)
  9. Lucia (Agency Administrator)
  10. Joel (Agency Administrator)
  11. Ben (EPA Representative)
  12. John (State DEQ Representative)
  13. Joe (Unified Incident Command Spokesperson)

**Summary of the Meeting:**

The meeting is a planning meeting for the operational period ahead, focusing on the response to a tractor-trailer accident that has spilled diazonon and diesel fuel. The Operations Section Chief, Rich, presents the plan for the night operational period, which includes improving dikes around the spill area, conducting a safety analysis, and setting up a responder rehab area. The team discusses logistics, liaison, information, and finance, and receives updates from various agencies. The meeting concludes with a final buy-off from the incident management team members and stakeholders, confirming the support for the operational plan.

**Key Discussion Points:**

1. The incident objectives were confirmed as still valid.
2. The Operations Section Chief presented the plan for the night operational period.
3. The safety analysis highlighted the need for proper PPE, mitigation measures, and concern about fatigue and dehydration.
4. Logistics discussed food, drinks, and tents for the Responder Rehab Area.
5. Liaison and Information updates were provided.
6. Finance discussed tracking expenditures and traffic control safety devices.
7. The team supported the plan with a safety buy-off.

**Takeaways:**

1. The incident response plan is in place, with a focus on improving dikes around the spill area.
2. Safety concerns were addressed, including proper PPE and mitigation measures.
3. Logistics and information updates were provided.
4. The team supported the plan with a safety buy-off.
5. The Unified Incident Command Spokesperson will continue to represent the team for the rest of the operational period.

**List of Action Items:**

1. IAP components should be submitted to the Incident Commander by 1530 this afternoon.
2. Logistics should build a briefing area north of the command post.
3. The operational briefing for tonight should be held at 1800.
4. A 2000 planning meeting will be held to work on tomorrow's day operations.
5. Agency administrators should be notified of the plan and its details.
6. The Incident Commander should receive updates on the IAP components and logistics.
7. The team should be prepared for the operational briefing and the 2000 planning meeting.

In [13]:
with open("/content/drive/MyDrive/ai-projects/quick-meet/transcript.md", "w") as f:
    f.write(final_script)